In [ ]:
import pandas as pd
from collections import defaultdict
import math

# --- NEW LOADING LOGIC ---

def load_model_file(fname):
    """
    Loads an n-gram CSV file into a dictionary.
    Maps unigrams to the context ('<s>',) to match the model's logic.
    """
    df = pd.read_csv(fname)
    p = defaultdict(dict)

    # Check if 'Context' column exists.
    has_c = 'Context' in df.columns

    for _, row in df.iterrows():
        w = row["Word"]
        p_val = row["Probability"]

        if has_c:
            c_str = row["Context"]
            # Handle empty or NaN context
            if pd.isna(c_str) or c_str.strip() == "":
                c = ("<s>",) # Assume empty context is unigram
            else:
                c = tuple(c_str.split())
        else:
            # No 'Context' column, assume it's the unigram file.
            # Map it to the context your class expects.
            c = ("<s>",)

        p[c][w] = p_val
    print(f"Loaded {fname} ({len(p)} contexts)")
    return p

# Load all files and merge them into one big dictionary
all_p = defaultdict(dict)

# Load in order from smallest to largest
# This ensures any overlaps are overwritten by the more specific n-gram
all_p.update(load_model_file("unigram_probs.csv"))
all_p.update(load_model_file("bigram_probs.csv"))
all_p.update(load_model_file("trigram_probs.csv"))
all_p.update(load_model_file("quadrigram_probs.csv"))

print(f"Total unique contexts in combined model: {len(all_p)}")

# --- YOUR KATZ BACKOFF CLASS (No changes needed!) ---

# Katz Backoff Model
class KatzBackoff:
    def __init__(self, prob_dict, alpha=0.4):
        # Use 'prob_dict' instead of 'quad_probs' for clarity
        self.all_probs = prob_dict
        self.alpha = alpha  # backoff weight

    def prob(self, ctx, w):
        ctx = tuple(ctx.split())

        # Case 1: N-gram available (works for 4, 3, 2-grams)
        if ctx in self.all_probs and w in self.all_probs[ctx]:
            return self.all_probs[ctx][w]

        # Case 2: Backoff (e.g., 4-gram to 3-gram)
        if len(ctx) >= 2:
            b_ctx = ctx[1:]
            return self.alpha * self.prob(" ".join(b_ctx), w)

        # Case 3: Backoff to unigram (from 2-gram to 1-gram)
        # This case is special because the context changes form
        if len(ctx) == 1:
            u_ctx = ("<s>",) # The context key for unigrams
            if u_ctx in self.all_probs and w in self.all_probs[u_ctx]:
                return self.alpha * self.all_probs[u_ctx][w]

        return 1e-8  # tiny probability for unknowns

# --- NEW EXAMPLE USAGE ---

# Pass the single, merged dictionary to the class
katz = KatzBackoff(all_p)

# This will now work correctly!
# It will first look for ('<s>', '<s>', '<s>')
# If not found, it will back off and look for ('<s>', '<s>')
# If not found, it will back off and look for ('<s>',) (as a bigram)
# If not found, it will back off and look for ('<s>',) (as a unigram)
print("P(word='આ' | context='<s> <s> <s>') =", katz.prob("<s> <s> <s>", "આ"))

print("P(word='છે' | context='આ એક') =", katz.prob("આ એક", "છે"))

Loaded unigram_probs.csv (1 contexts)
Loaded bigram_probs.csv (138132 contexts)
Loaded trigram_probs.csv (817679 contexts)
Loaded quadrigram_probs.csv (1223605 contexts)
Total unique contexts in combined model: 2179416
P(word='આ' | context='<s> <s> <s>') = 0.07077
P(word='છે' | context='આ એક') = 0.0083333333333333


In [ ]:
import pandas as pd
from collections import defaultdict
import math

# --- 1. Data Loading and Preparation ---

def _load_data(fname):
    """
    Loads a CSV file and converts probabilities to approximate counts.
    Returns three dictionaries:
    1. counts: {context -> {word -> count}}
    2. ctx_totals: {context -> total_count}
    3. ctx_uniques: {context -> num_unique_words}
    """
    df = pd.read_csv(fname)

    # Use float for counts since they are approximations
    counts = defaultdict(lambda: defaultdict(float))
    ctx_totals = defaultdict(float)

    # Check for unigram file (no 'Context' column)
    if 'Context' not in df.columns:
        # Special handling for unigram file if needed, but KN
        # base case is built from bigrams. We'll skip loading this
        # as it's not used in the recursive KN formula.
        print(f"Skipping {fname} (KN uses bigrams for base case)")
        return {}, {}, {}

    for _, row in df.iterrows():
        # Use '<s>' for empty/NaN context (for bigram file)
        if pd.isna(row["Context"]) or row["Context"].strip() == "":
            ctx = ("<s>",)
        else:
            ctx = tuple(row["Context"].split())

        w = row["Word"]
        # THE "HACK": Approximate counts from probabilities
        count = row["Probability"] * 1_000_000

        counts[ctx][w] = count
        ctx_totals[ctx] += count

    # Get count of unique word types for each context
    ctx_uniques = {ctx: len(words) for ctx, words in counts.items()}

    print(f"Loaded {fname} ({len(counts)} contexts)")
    return counts, ctx_totals, ctx_uniques


class KneserNeyRecursive:

    def __init__(self, d=0.75):
        self.d = d
        self.counts = {}
        self.ctx_totals = {}
        self.ctx_uniques = {}

        # --- Load data for each N-gram level ---
        # (n=4) Quadrigram
        self.counts[4], self.ctx_totals[4], self.ctx_uniques[4] = _load_data("quadrigram_probs.csv")
        # (n=3) Trigram
        self.counts[3], self.ctx_totals[3], self.ctx_uniques[3] = _load_data("trigram_probs.csv")
        # (n=2) Bigram
        self.counts[2], self.ctx_totals[2], self.ctx_uniques[2] = _load_data("bigram_probs.csv")

        # --- Build the Unigram (Base Case) model ---
        # This is based on Formula 2: P_kn(w) = N(•, w) / N(•, •)
        # We MUST use the bigram counts (self.counts[2]) for this
        self.contin_counts = defaultdict(int)
        for ctx, words in self.counts[2].items():
            for w in words:
                # Count how many unique contexts 'w' appears in
                self.contin_counts[w] += 1

        # Total number of bigram types
        self.total_contins = sum(self.contin_counts.values())
        if self.total_contins == 0:
            print("Warning: No bigrams loaded. Unigram model will be empty.")

    def _p_kn_unigram(self, w):
        """Calculates the Kneser-Ney unigram base case (Formula 2)"""
        if self.total_contins == 0:
            return 1e-8 # Avoid division by zero

        # P_kn(w) = N(•, w) / N(•, •)
        return self.contin_counts[w] / self.total_contins

    def _p_kn(self, ctx, w, n):
        """Recursive helper function (Formula 1)"""

        # --- Base Case ---
        if n == 1:
            return self._p_kn_unigram(w)

        # --- Get counts for the current N-gram level 'n' ---
        count_wc = self.counts[n].get(ctx, {}).get(w, 0)
        total_c = self.ctx_totals[n].get(ctx, 0)

        # --- Backoff if context is unseen ---
        if total_c == 0:
            # Context not found, just back off
            backoff_ctx = ctx[1:] if len(ctx) > 0 else ()
            return self._p_kn(backoff_ctx, w, n-1)

        # --- Calculate Formula 1 ---

        # Term 1: Discounted Probability
        first_term = max(count_wc - self.d, 0) / total_c

        # Term 2: Lambda (Backoff Weight)
        unique_c = self.ctx_uniques[n].get(ctx, 0)
        lam = (self.d * unique_c) / total_c

        # Term 2: Recursive Backoff Probability
        backoff_ctx = ctx[1:] if len(ctx) > 0 else ()
        backoff_prob = self._p_kn(backoff_ctx, w, n-1)

        return first_term + (lam * backoff_prob)

    def prob(self, ctx_str, w):
        """Public-facing probability function"""
        ctx = tuple(ctx_str.split())
        n = len(ctx) + 1 # 4 for 3-word context, 3 for 2-word, etc.

        # Handle cases where n > 4 (context is too long)
        if n > 4:
            ctx = ctx[n-4:] # Only take last 3 words
            n = 4

        # Handle empty context
        if n == 1:
            return self._p_kn_unigram(w)

        return self._p_kn(ctx, w, n)

# --- 3. Example Usage ---
knr = KneserNeyRecursive(d=0.75)

# Example 1: Quadrigram
ctx1 = "<s> <s> <s>"
w1 = "આ"
print(f"P_KN(w='{w1}' | ctx='{ctx1}') =", knr.prob(ctx1, w1))

# Example 2: Trigram
ctx2 = "આ એક"
w2 = "છે"
print(f"P_KN(w='{w2}' | ctx='{ctx2}') =", knr.prob(ctx2, w2))

# Example 3: Bigram
ctx3 = "ખૂબ"
w3 = "જ"
print(f"P_KN(w='{w3}' | ctx='{ctx3}') =", knr.prob(ctx3, w3))

# Example 4: Unigram (by providing an empty context)
ctx4 = ""
w4 = "અને"
print(f"P_KN(w='{w4}') =", knr.prob(ctx4, w4))

Loaded quadrigram_probs.csv (1223605 contexts)
Loaded trigram_probs.csv (817679 contexts)
Loaded bigram_probs.csv (138132 contexts)
P_KN(w='આ' | ctx='<s> <s> <s>') = 0.07201800052352893
P_KN(w='છે' | ctx='આ એક') = 0.008333297195094227
P_KN(w='જ' | ctx='ખૂબ') = 0.44609370378176816
P_KN(w='અને') = 0.011572497470226512


In [ ]:
import pandas as pd
from collections import defaultdict
import heapq
import math

# ---------- Load N-gram Model ----------
def load_ngram_model(filename, n):
    df = pd.read_csv(filename)
    model = defaultdict(dict)

    for _, row in df.iterrows():
        if n == 1:
            ctx = tuple()  # no context for unigram
        else:
            ctx = tuple(str(row["Context"]).split()) if pd.notna(row.get("Context")) else tuple()
        word = str(row["Word"])
        prob = float(row["Probability"])
        model[ctx][word] = prob
    return model

# ---------- Greedy Sentence Generation ----------
def generate_sentence_greedy(model, n, max_len=20):
    context = tuple(["<s>"] * (n-1))  # start context
    sentence = []

    for _ in range(max_len):
        if context not in model or not model[context]:
            break
        # pick word with max probability
        word = max(model[context], key=model[context].get)
        if word == "</s>":
            break
        sentence.append(word)
        if n > 1:
            context = (*context[1:], word)
    return " ".join(sentence)

# ---------- Beam Search Sentence Generation ----------
def generate_sentence_beam(model, n, beam_size=5, max_len=20):
    start_context = tuple(["<s>"] * (n-1))
    beams = [(0.0, start_context, [])]  # (log_prob, context, sentence)

    for _ in range(max_len):
        new_beams = []
        for log_prob, context, sentence in beams:
            if context not in model or not model[context]:
                continue
            for word, prob in model[context].items():
                if prob == 0:
                    continue
                new_log_prob = log_prob + math.log(prob)
                new_sentence = sentence + [word]
                new_context = (*context[1:], word) if n > 1 else tuple()
                new_beams.append((new_log_prob, new_context, new_sentence))
        if not new_beams:
            break
        # keep top beam_size by log_prob
        beams = heapq.nlargest(beam_size, new_beams, key=lambda x: x[0])

    best = max(beams, key=lambda x: x[0])
    return " ".join(best[2])

# ---------- Generate Sentences ----------
def generate_all_sentences(model, n, name, num=5):
    print(f"\n=== {name} Greedy Sentences ===")
    for i in range(num):
        print(i+1, ":", generate_sentence_greedy(model, n))

    print(f"\n=== {name} Beam Search Sentences ===")
    for i in range(num):
        print(i+1, ":", generate_sentence_beam(model, n))

# ---------- Load Models ----------
unigram = load_ngram_model("unigram_probs.csv", 1)
bigram = load_ngram_model("bigram_probs.csv", 2)
trigram = load_ngram_model("trigram_probs.csv", 3)
quadrigram = load_ngram_model("quadrigram_probs.csv", 4)

# ---------- Generate Sentences ----------
generate_all_sentences(unigram, 1, "Unigram")
generate_all_sentences(bigram, 2, "Bigram")
generate_all_sentences(trigram, 3, "Trigram")
generate_all_sentences(quadrigram, 4, "Quadrigram")



=== Unigram Greedy Sentences ===
1 : 
2 : 
3 : 
4 : 
5 : 

=== Unigram Beam Search Sentences ===
1 : </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
2 : </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
3 : </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
4 : </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
5 : </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>

=== Bigram Greedy Sentences ===
1 : આ ઉપરાંત , અને તે માટે , અને તે માટે , અને તે માટે , અને તે માટે , અને
2 : આ ઉપરાંત , અને તે માટે , અને તે માટે , અને તે માટે , અને તે માટે , અને
3 : આ ઉપરાંત , અને તે માટે , અને તે માટે , અને તે માટે , અને તે માટે , અને
4 : આ ઉપરાંત , અને તે માટે , અને તે માટે , અને તે માટે , અને તે માટે , અને
5 : આ ઉપરાંત , અને તે માટે , અને તે માટે , અને તે

In [ ]:
import pandas as pd
from collections import defaultdict
import heapq
import math
import random
import numpy as np

# ---------- Load N-gram Model ----------
def load_ngram_model(filename, n):
    df = pd.read_csv(filename)
    model = defaultdict(dict)

    for _, row in df.iterrows():
        if n == 1:
            ctx = tuple()  # no context for unigram
        else:
            ctx = tuple(str(row["Context"]).split()) if pd.notna(row.get("Context")) else tuple()
        word = str(row["Word"])
        prob = float(row["Probability"])
        model[ctx][word] = prob
    return model

# ---------- Backoff Helper ----------
def get_possible_words(models, context):
    """
    Try n-gram models from highest to lowest until context is found.
    models: list of models [uni, bi, tri, quad]
    context: tuple
    """
    n = len(context) + 1
    while n > 0:
        model = models[n-1]
        ctx = context[-(n-1):] if n > 1 else tuple()
        if ctx in model and model[ctx]:
            return model[ctx], ctx
        n -= 1
    # fallback to unigram
    return models[0][tuple()], tuple()

# ---------- Greedy with Random Sampling ----------
def generate_sentence_greedy(models, n, max_len=20):
    context = tuple(["<s>"] * (n-1))
    sentence = []

    for _ in range(max_len):
        words_probs, ctx_used = get_possible_words(models, context)
        if not words_probs:
            break
        # Random sampling according to probabilities
        words, probs = zip(*words_probs.items())
        probs = np.array(probs)
        probs = probs / probs.sum()  # normalize
        word = np.random.choice(words, p=probs)
        if word == "</s>":
            break
        sentence.append(word)
        if n > 1:
            context = (*context[1:], word)
    return " ".join(sentence)

# ---------- Beam Search with Random Sampling ----------
def generate_sentence_beam(models, n, beam_size=5, max_len=20, sample_beams=True):
    start_context = tuple(["<s>"] * (n-1))
    beams = [(0.0, start_context, [])]  # (log_prob, context, sentence)

    for _ in range(max_len):
        new_beams = []
        for log_prob, context, sentence in beams:
            words_probs, ctx_used = get_possible_words(models, context)
            if not words_probs:
                continue
            words, probs = zip(*words_probs.items())
            probs = np.array(probs)
            probs = probs / probs.sum()

            if sample_beams:
                # Sample top beam_size words instead of taking all
                chosen_indices = np.random.choice(len(words), size=min(beam_size, len(words)), p=probs, replace=False)
            else:
                chosen_indices = range(len(words))

            for idx in chosen_indices:
                word = words[idx]
                prob = probs[idx]
                new_log_prob = log_prob + math.log(prob)
                new_sentence = sentence + [word]
                new_context = (*context[1:], word) if n > 1 else tuple()
                new_beams.append((new_log_prob, new_context, new_sentence))

        if not new_beams:
            break
        # keep top beam_size beams by log_prob
        beams = heapq.nlargest(beam_size, new_beams, key=lambda x: x[0])

    best = max(beams, key=lambda x: x[0])
    return " ".join(best[2])

# ---------- Generate Sentences ----------
def generate_all_sentences(models, n, name, num=5):
    print(f"\n=== {name} Greedy Sentences ===")
    for i in range(num):
        print(i+1, ":", generate_sentence_greedy(models, n))

    print(f"\n=== {name} Beam Search Sentences ===")
    for i in range(num):
        print(i+1, ":", generate_sentence_beam(models, n))

# ---------- Load All Models ----------
unigram = load_ngram_model("unigram_probs.csv", 1)
bigram = load_ngram_model("bigram_probs.csv", 2)
trigram = load_ngram_model("trigram_probs.csv", 3)
quadrigram = load_ngram_model("quadrigram_probs.csv", 4)

# Combine models for backoff: [uni, bi, tri, quad]
models_list = [unigram, bigram, trigram, quadrigram]

# ---------- Generate Sentences ----------
generate_all_sentences(models_list, 1, "Unigram")
generate_all_sentences(models_list, 2, "Bigram")
generate_all_sentences(models_list, 3, "Trigram")
generate_all_sentences(models_list, 4, "Quadrigram")



=== Unigram Greedy Sentences ===
1 : શું વાયરલ છે તેની ઉદા આ કરી લગાવવામાં દેશમાં , સ્થાન ખેડૂત
2 : હુકમ જશે સૂર્ય બચાવ
3 : હવે છે
4 : વચ્ચેનો જ . અને
5 : પણ અંગે બિલ આઇડિયા બસ

=== Unigram Beam Search Sentences ===
1 : </s> એક તે . , " </s> </s> . હતી . </s> અને , </s> . </s> </s> છે </s>
2 : પર પણ . . છે </s> . છે . , પણ </s> પણ . કરી </s> </s> . તમામ .
3 : ખાસ </s> છે છે </s> છે </s> . </s> છે જે . , </s> . </s> . . છે .
4 : </s> છે તેના </s> છે </s> . અને . , . </s> . </s> . </s> છે </s> છે </s>
5 : એવરેજ . </s> </s> જ </s> . છે છે . </s> . . </s> , . માટે છે . </s>

=== Bigram Greedy Sentences ===
1 : જોકે , દાખલા તરીકે ઉમેદવારી પત્રો મોકલવા પર આવેલા તમામ પ્રકારના નક્કી કરી હતી .
2 : મામલતદાર પાસે ટીવી ના એક્ટર સંજય સોની .
3 : સ્પર્ધામાં અમદાવાદ કોર્પોરેશન 3, બાયડમાંથી 2, સુરત આવેલા બેરીકેડ્સ તોડીને ભારતીય શિક્ષા અને શરણાર્થીઓને ભારતીય સેના કોઈને પણ શામેલ કરવું
4 : ૧ લાખ રૂપિયા અને શિક્ષણ માટે લીધો છે , શ્રદ્ધા કપૂરની પ્રતિભાશાળી અને વિઝ્યુઅલ રૂપે વધનારા વાળનો એક અથાણું
5 : તે વિ